In [ ]:
import random
import time
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

## Modalidad Juego 1

In [ ]:
class Jugador():
    def __init__(self,nombre):
        self.nombre=nombre
        self.paredes={
            'Rojo':0,
            'Azul':0,
            'Verde':0,
            'Amarillo':0,
            'Violeta':0}
        self.turno=True
        self.turnos_perdidos=0
    
    def tirar_dado(self):
        
        if self.turno==False:
            print("No es tu turno")
            self.turno=True
            self.turnos_perdidos+=1
            return 
            
        else:
            colores = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
            color_seleccionado = random.choice(colores)
            print(f"El color que salió es: {color_seleccionado}")
            
            if color_seleccionado!='Rojo' and color_seleccionado!='Negro':
                if self.paredes[color_seleccionado]==0:
                    self.paredes[color_seleccionado]=1
                else:
                    self.turno=False
                
            elif color_seleccionado=='Negro':
                colores_disponibles = [color for color in ["Azul", "Verde", "Amarillo", "Violeta"] if self.paredes[color] == 1]
                if colores_disponibles:
                    color_eliminar = random.choice(colores_disponibles)
                    self.paredes[color_eliminar] = 0
                    print(f"El color {color_eliminar} ha sido eliminado.")
                else:
                    print("No hay colores con valor 1 para eliminar.")
                
            elif color_seleccionado=='Rojo' and sum(self.paredes.values())==4:
                self.paredes[color_seleccionado]=1
        return color_seleccionado
    
    def resetear(self):
        self.paredes = {
            'Rojo': 0,
            'Azul': 0,
            'Verde': 0,
            'Amarillo': 0,
            'Violeta': 0
        }
        self.turno = True
        self.turnos_perdidos = 0
        
class Juego():
    def __init__(self):
        self.jugadores = []
        self.turno = 0
        self.tiempo=0
        self.ganador=None
        self.rondas=0
        self.turnos_perdidos=0
    
    def añadir_jugador(self,jugador):
        self.jugadores.append(jugador)
    
    def jugar(self):
        for jugador in self.jugadores:
            jugador.resetear()
        
        self._resetear()
            
        jugando=True
        while jugando:
            self.rondas+=1
            for jugador in self.jugadores:
                jugador.tirar_dado()
                self.tiempo+=random.randint(10,20)
                self.turno+=1
                if sum(jugador.paredes.values())==5:
                    print(f"El jugador {jugador.nombre} ha ganado")
                    jugando=False
                    self.ganador=jugador
                    break
        for jug in self.jugadores:
            self.turnos_perdidos+=jug.turnos_perdidos
        self._estadisticas_juego()
        
    def _estadisticas_juego(self):
        print(f"El juego duró {self.tiempo/60} minutos")
        print(f"El ganador fue {self.ganador.nombre}")
        print(f"El jugador {self.ganador.nombre} perdió {self.ganador.turnos_perdidos} turnos")
        print(f"El juego duró {self.rondas} rondas")
        print(f"Se perdieron {self.turnos_perdidos} turnos")
    
    def _resetear(self):
        self.turno = 0
        self.tiempo = 0
        self.ganador = None
        self.rondas = 0
        self.turnos_perdidos = 0

### Resultados

In [ ]:
if __name__ == "__main__":
    juegos_modalidad1=pd.DataFrame(columns=['Cantidad de jugadores','Tiempo','Ganador','Turnos Totales','Rondas','Turnos Perdidos Totales'])
    # Cantidad de jugadores
    cantidad_jugadores = [2,4,10,50,100,500]
    for cant in cantidad_jugadores:
        juego = Juego()
        
        for j in range(cant):
            jugador = Jugador(f"Jugador {j+1}")
            juego.añadir_jugador(jugador)
            
        for i in range(30):
            juego.jugar()
            simulacion=pd.DataFrame({'Cantidad de jugadores':len(juego.jugadores),'Tiempo':juego.tiempo/60,'Ganador':juego.ganador.nombre,'Turnos Totales':juego.turno,'Rondas':juego.rondas,'Turnos Perdidos Totales':juego.turnos_perdidos},index=[0])
            juegos_modalidad1=pd.concat([juegos_modalidad1,simulacion],ignore_index=True)
    juegos_modalidad1[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']]=juegos_modalidad1[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']].astype(int)

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, col in enumerate(['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=juegos_modalidad1, x=col, hue='Cantidad de jugadores', multiple='stack', palette='muted', ax=ax)
    ax.set_title(f'Histograma de {col}')

plt.tight_layout()
plt.show()

In [ ]:
juegos_modalidad1.groupby('Cantidad de jugadores').agg({'Rondas':['mean','std'],'Tiempo':['mean','std'],'Turnos Perdidos Totales':['mean','std'],'Turnos Totales':['mean','std']}).reset_index()

In [ ]:
572/60

In [ ]:
import scipy.stats as stats

juegos_modalidad1.groupby('Cantidad de jugadores').agg({'Rondas':['mean','std'],'Tiempo':['mean','std'],'Turnos Perdidos Totales':['mean','std'],'Turnos Totales':['mean','std']}).reset_index()
modalidad1=pd.DataFrame(columns=['Cantidad de jugadores','Tiempo','Ganador','Turnos Totales','Rondas','Turnos Perdidos Totales'])
intervalos=pd.DataFrame(columns=['Cantidad de jugadores','Muestra','Limite Inferior','Limite Superior'])
cantidad_jugadores = [2,4,10,50,100,500]
for cant in cantidad_jugadores:
    muestras_piloto = juegos_modalidad1[juegos_modalidad1['Cantidad de jugadores']==cant]['Rondas'] 
    media = np.mean(muestras_piloto)
    desvio = np.std(muestras_piloto, ddof=1)
    alpha=0.05

    precision = 0.01 * media
    Z = stats.norm.ppf(1-alpha/2) 

    n = (Z * desvio / precision) ** 2
    n = int(np.ceil(n)) 

    hw=Z*desvio/np.sqrt(n)
    while hw>precision:
        n+=1
        hw=Z*desvio/np.sqrt(n)
    print(f"Tamaño óptimo de muestra: {n},{cant} jugadores")
    
    juego = Juego()
    
    for j in range(cant):
        jugador = Jugador(f"Jugador {j+1}")
        juego.añadir_jugador(jugador)
        
    for i in range(n):
        juego.jugar()
        simulacion=pd.DataFrame({'Cantidad de jugadores':len(juego.jugadores),'Tiempo':juego.tiempo/60,'Ganador':juego.ganador.nombre,'Turnos Totales':juego.turno,'Rondas':juego.rondas,'Turnos Perdidos Totales':juego.turnos_perdidos},index=[0])
        modalidad1=pd.concat([modalidad1,simulacion],ignore_index=True)
    intervalos=pd.concat([intervalos,pd.DataFrame({'Cantidad de jugadores':cant,'Muestra':n,'Limite Inferior':media-hw,'Limite Superior':media+hw},index=[0])],ignore_index=True)    
modalidad1[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']]=modalidad1[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']].astype(int)

In [ ]:
modalidad1.groupby('Cantidad de jugadores')['Rondas'].count()

In [ ]:
modalidad1.groupby('Cantidad de jugadores')['Rondas'].count()

In [ ]:
modalidad1.groupby('Cantidad de jugadores').agg({'Rondas':['mean','std'],'Tiempo':['mean','std'],'Turnos Totales':['mean','std'],'Turnos Perdidos Totales':['mean','std']}).reset_index().reset_index()

In [ ]:
intervalos

In [ ]:
li=media-hw
ls=media+hw 
print(f"El intervalo de confianza es: ({li},{ls})")

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, col in enumerate(['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=modalidad1, x=col, hue='Cantidad de jugadores', multiple='stack', palette='muted', ax=ax)
    ax.set_title(f'Histograma de {col}')

plt.tight_layout()
plt.show()

In [ ]:
sns.scatterplot(data=juegos_modalidad1,x='Rondas',y='Tiempo',hue='Cantidad de jugadores',palette='muted',s=30)

In [ ]:
sns.scatterplot(data=juegos_modalidad1,x='Turnos Totales',y='Turnos Perdidos Totales',hue='Cantidad de jugadores',palette='muted',s=30)

In [ ]:
sns.boxplot(data=juegos_modalidad1,x='Cantidad de jugadores',y='Rondas',palette='muted')

## Modalidad Juego 2

Si un jugador saca el negro pierde todas sus paredes.

In [ ]:
class Jugador():
    def __init__(self,nombre):
        self.nombre=nombre
        self.paredes={
            'Rojo':0,
            'Azul':0,
            'Verde':0,
            'Amarillo':0,
            'Violeta':0}
        self.turno=True
        self.turnos_perdidos=0
    
    def tirar_dado(self):
        
        if self.turno==False:
            self.turno=True
            self.turnos_perdidos+=1
            return 
            
        else:
            colores = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
            color_seleccionado = random.choice(colores)
            
            if color_seleccionado!='Rojo' and color_seleccionado!='Negro':
                if self.paredes[color_seleccionado]==0:
                    self.paredes[color_seleccionado]=1
                else:
                    self.turno=False
                
            elif color_seleccionado=='Negro':
                colores_disponibles = [color for color in ["Azul", "Verde", "Amarillo", "Violeta"] if self.paredes[color] == 1]
                if colores_disponibles:
                    for color_eliminar in colores_disponibles:
                        self.paredes[color_eliminar] = 0

                
            elif color_seleccionado=='Rojo' and sum(self.paredes.values())==4:
                self.paredes[color_seleccionado]=1
        return color_seleccionado
    
    def resetear(self):
        self.paredes = {
            'Rojo': 0,
            'Azul': 0,
            'Verde': 0,
            'Amarillo': 0,
            'Violeta': 0
        }
        self.turno = True
        self.turnos_perdidos = 0
        
class Juego():
    def __init__(self):
        self.jugadores = []
        self.turno = 0
        self.tiempo=0
        self.ganador=None
        self.rondas=0
        self.turnos_perdidos=0
    
    def añadir_jugador(self,jugador):
        self.jugadores.append(jugador)
    
    def jugar(self):
        for jugador in self.jugadores:
            jugador.resetear()
        
        self._resetear()
            
        jugando=True
        while jugando:
            self.rondas+=1
            for jugador in self.jugadores:
                jugador.tirar_dado()
                self.tiempo+=random.randint(10,20)
                self.turno+=1
                if sum(jugador.paredes.values())==5:
                    jugando=False
                    self.ganador=jugador
                    break
        for jug in self.jugadores:
            self.turnos_perdidos+=jug.turnos_perdidos
    
    def _resetear(self):
        self.turno = 0
        self.tiempo = 0
        self.ganador = None
        self.rondas = 0
        self.turnos_perdidos = 0

### Resultados

In [ ]:
if __name__ == "__main__":
    juegos_modalidad2=pd.DataFrame(columns=['Cantidad de jugadores','Tiempo','Ganador','Turnos Totales','Rondas','Turnos Perdidos Totales'])
    # Cantidad de jugadores
    cantidad_jugadores = [2,4,10,50,100,500]
    for cant in cantidad_jugadores:
        juego = Juego()
        
        for j in range(cant):
            jugador = Jugador(f"Jugador {j+1}")
            juego.añadir_jugador(jugador)
            
        for i in range(30):
            juego.jugar()
            simulacion2=pd.DataFrame({'Cantidad de jugadores':len(juego.jugadores),'Tiempo':juego.tiempo/60,'Ganador':juego.ganador.nombre,'Turnos Totales':juego.turno,'Rondas':juego.rondas,'Turnos Perdidos Totales':juego.turnos_perdidos},index=[0])
            juegos_modalidad2=pd.concat([juegos_modalidad2,simulacion2],ignore_index=True)
    juegos_modalidad2[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']]=juegos_modalidad2[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']].astype(int)

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, col in enumerate(['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=juegos_modalidad2, x=col, hue='Cantidad de jugadores', multiple='stack', palette='muted', ax=ax)
    ax.set_title(f'Histograma de {col}')

plt.tight_layout()
plt.show()

In [ ]:
juegos_modalidad2.groupby('Cantidad de jugadores').agg({'Rondas':['mean','std'],'Tiempo':['mean','std'],'Turnos Totales':['mean','std'],'Turnos Perdidos Totales':['mean','std']}).reset_index()


In [ ]:
import scipy.stats as stats

modalidad2=pd.DataFrame(columns=['Cantidad de jugadores','Tiempo','Ganador','Turnos Totales','Rondas','Turnos Perdidos Totales'])
intervalos2=pd.DataFrame(columns=['Cantidad de jugadores','Muestra','Limite Inferior','Limite Superior'])
cantidad_jugadores = [2,4,10,50,100,500]
for cant in cantidad_jugadores:
    muestras_piloto = juegos_modalidad2[juegos_modalidad2['Cantidad de jugadores']==cant]['Rondas'] 
    media = np.mean(muestras_piloto)
    desvio = np.std(muestras_piloto, ddof=1)
    alpha=0.05

    precision = 0.01 * media
    Z = stats.norm.ppf(1-alpha/2) 

    n = (Z * desvio / precision) ** 2
    n = int(np.ceil(n)) 

    hw=Z*desvio/np.sqrt(n)
    while hw>precision:
        n+=1
        hw=Z*desvio/np.sqrt(n)
    print(f"Tamaño óptimo de muestra: {n},{cant} jugadores")
    
    juego = Juego()
    
    for j in range(cant):
        jugador = Jugador(f"Jugador {j+1}")
        juego.añadir_jugador(jugador)
        
    for i in range(n):
        juego.jugar()
        simulacion2=pd.DataFrame({'Cantidad de jugadores':len(juego.jugadores),'Tiempo':juego.tiempo/60,'Ganador':juego.ganador.nombre,'Turnos Totales':juego.turno,'Rondas':juego.rondas,'Turnos Perdidos Totales':juego.turnos_perdidos},index=[0])
        modalidad2=pd.concat([modalidad2,simulacion2],ignore_index=True)
    intervalos2=pd.concat([intervalos2,pd.DataFrame({'Cantidad de jugadores':cant,'Muestra':n,'Limite Inferior':media-hw,'Limite Superior':media+hw},index=[0])],ignore_index=True)    
modalidad2[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']]=modalidad2[['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales']].astype(int)

In [ ]:
intervalos2

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, col in enumerate(['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=modalidad2, x=col, hue='Cantidad de jugadores', multiple='stack', palette='muted', ax=ax)
    ax.set_title(f'Histograma de {col}')

plt.tight_layout()
plt.show()

In [ ]:
sns.scatterplot(data=modalidad2,x='Rondas',y='Tiempo',hue='Cantidad de jugadores',palette='muted',s=30)

## Modalidad Juego 3

Que pasa si cuando saco Negro le robo a otra persona una pared aleatoria? Tiro el dado para ver que pared le saco, y si ya tengo esa pared pierdo todas las paredes

In [ ]:
class Jugador():
    def __init__(self,nombre):
        self.nombre=nombre
        self.paredes={
            'Rojo':0,
            'Azul':0,
            'Verde':0,
            'Amarillo':0,
            'Violeta':0}
        self.turno=True
        self.turnos_perdidos=0
    
    def tirar_dado(self,jugador):
        
        if self.turno==False:
            self.turno=True
            self.turnos_perdidos+=1
            return 
            
        else:
            colores = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
            color_seleccionado = random.choice(colores)
            
            if color_seleccionado!='Rojo' and color_seleccionado!='Negro':
                if self.paredes[color_seleccionado]==0:
                    self.paredes[color_seleccionado]=1
                else:
                    self.turno=False
                
            elif color_seleccionado=='Negro':
                self._robar_pared(jugador)

            elif color_seleccionado=='Rojo' and sum(self.paredes.values())==4:
                self.paredes[color_seleccionado]=1
        return color_seleccionado
    
    def _robar_pared(self,jugador):
        colores = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
        color_seleccionado = random.choice(colores)
        
        if color_seleccionado not in ['Negro','Rojo'] and self.paredes[color_seleccionado]==0:
            if color_seleccionado in [color for color in ["Azul", "Verde", "Amarillo", "Violeta"] if jugador.paredes[color] == 1]:
                self.paredes[color_seleccionado]=1
                jugador.paredes[color_seleccionado]=0
        else:
            colores_disponibles = [color for color in ["Azul", "Verde", "Amarillo", "Violeta"] if self.paredes[color] == 1]
            if colores_disponibles:
                for color_eliminar in colores_disponibles:
                    self.paredes[color_eliminar] = 0
        
    def resetear(self):
        self.paredes = {
            'Rojo': 0,
            'Azul': 0,
            'Verde': 0,
            'Amarillo': 0,
            'Violeta': 0
        }
        self.turno = True
        self.turnos_perdidos = 0
        
class Juego():
    def __init__(self):
        self.jugadores = []
        self.turno = 0
        self.tiempo=0
        self.ganador=None
        self.rondas=0
        self.turnos_perdidos=0
    
    def añadir_jugador(self,jugador):
        self.jugadores.append(jugador)
    
    def jugar(self):
        for jugador in self.jugadores:
            jugador.resetear()
        
        self._resetear()
            
        jugando=True
        while jugando:
            self.rondas+=1
            jugador_anterior=None
            for jugador in self.jugadores:
                try:
                    jugador.tirar_dado(jugador_anterior)
                    self.turno+=1
                except AttributeError:
                    pass
                self.tiempo+=random.randint(10,40)
                jugador_anterior=jugador
                if sum(jugador.paredes.values())==5:
                    jugando=False
                    self.ganador=jugador
                    break
        for jug in self.jugadores:
            self.turnos_perdidos+=jug.turnos_perdidos
    
    def _resetear(self):
        self.turno = 0
        self.tiempo = 0
        self.ganador = None
        self.rondas = 0
        self.turnos_perdidos = 0

### Resultados

In [ ]:
if __name__ == "__main__":
    juego = Juego()

    # Cantidad de jugadores
    for i in range(4):
        jugador = Jugador(f"Jugador {i+1}")
        juego.añadir_jugador(jugador)
        
    juegos_modalida3=pd.DataFrame(columns=['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales'])

    # Cantidad de juegos
    for i in range(10000):
        juego.jugar()
        juegos_modalida3.loc[i]=[len(juego.jugadores),juego.tiempo/60,juego.turno,juego.rondas,juego.turnos_perdidos]

In [ ]:
juegos_modalida3.groupby('Cantidad de jugadores').agg({'Rondas':['mean','std'],'Tiempo':['mean','std'],'Turnos Totales':['mean','std'],'Turnos Perdidos Totales':['mean','std']}).reset_index()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, col in enumerate(['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=juegos_modalida3, x=col, kde=True, multiple='stack', palette='muted', ax=ax)
    ax.set_title(f'Histograma de {col}')

plt.tight_layout()
plt.show()

## Modalidad Juego 4

Si cuando saco una pared que ya tengo, vuelvo a tirar el dado y vuelvo a sacar el mismo color le puedo sacar una pared de ese color a otro jugador

In [ ]:
class Jugador():
    def __init__(self,nombre):
        self.nombre=nombre
        self.paredes={
            'Rojo':0,
            'Azul':0,
            'Verde':0,
            'Amarillo':0,
            'Violeta':0}
        self.turno=True
        self.turnos_perdidos=0
    
    def tirar_dado(self,jugador):
        
        if self.turno==False:
            self.turno=True
            self.turnos_perdidos+=1
            return 
            
        else:
            colores = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
            color_seleccionado = random.choice(colores)
            
            if color_seleccionado!='Rojo' and color_seleccionado!='Negro':
                if self.paredes[color_seleccionado]==0:
                    self.paredes[color_seleccionado]=1
                else:
                    colores_2 = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
                    color_seleccionado_2 = random.choice(colores_2)
                    if color_seleccionado_2==color_seleccionado:
                        if jugador.paredes[color_seleccionado_2]==1:
                            jugador.paredes[color_seleccionado_2]=0
                        else:
                            jugador.turno=False
                        
                    else:
                        self.turno=False
                
            elif color_seleccionado=='Negro':
                self._robar_pared(jugador)

            elif color_seleccionado=='Rojo' and sum(self.paredes.values())==4:
                self.paredes[color_seleccionado]=1
        return color_seleccionado
    
    def _robar_pared(self,jugador):
        colores = ["Rojo", "Azul", "Verde", "Amarillo", "Violeta", "Negro"]
        color_seleccionado = random.choice(colores)
        
        if color_seleccionado not in ['Negro','Rojo'] and self.paredes[color_seleccionado]==0:
            if color_seleccionado in [color for color in ["Azul", "Verde", "Amarillo", "Violeta"] if jugador.paredes[color] == 1]:
                self.paredes[color_seleccionado]=1
                jugador.paredes[color_seleccionado]=0
        else:
            colores_disponibles = [color for color in ["Azul", "Verde", "Amarillo", "Violeta"] if self.paredes[color] == 1]
            if colores_disponibles:
                for color_eliminar in colores_disponibles:
                    self.paredes[color_eliminar] = 0
        
    def resetear(self):
        self.paredes = {
            'Rojo': 0,
            'Azul': 0,
            'Verde': 0,
            'Amarillo': 0,
            'Violeta': 0
        }
        self.turno = True
        self.turnos_perdidos = 0
        
class Juego():
    def __init__(self):
        self.jugadores = []
        self.turno = 0
        self.tiempo=0
        self.ganador=None
        self.rondas=0
        self.turnos_perdidos=0
    
    def añadir_jugador(self,jugador):
        self.jugadores.append(jugador)
    
    def jugar(self):
        for jugador in self.jugadores:
            jugador.resetear()
        
        self._resetear()
            
        jugando=True
        while jugando:
            self.rondas+=1
            jugador_anterior=None
            for jugador in self.jugadores:
                try:
                    jugador.tirar_dado(jugador_anterior)
                    self.turno+=1
                except AttributeError:
                    pass
                self.tiempo+=random.randint(10,40)
                jugador_anterior=jugador
                if sum(jugador.paredes.values())==5:
                    jugando=False
                    self.ganador=jugador
                    break
        for jug in self.jugadores:
            self.turnos_perdidos+=jug.turnos_perdidos

    
    def _resetear(self):
        self.turno = 0
        self.tiempo = 0
        self.ganador = None
        self.rondas = 0
        self.turnos_perdidos = 0

### Resultados

In [ ]:
if __name__ == "__main__":
    juego = Juego()

    # Cantidad de jugadores
    for i in range(4):
        jugador = Jugador(f"Jugador {i+1}")
        juego.añadir_jugador(jugador)
        
    juegos_modalidad4=pd.DataFrame(columns=['Cantidad de jugadores','Tiempo','Turnos Totales','Rondas','Turnos Perdidos Totales'])

    # Cantidad de juegos
    for i in range(10000):
        juego.jugar()
        juegos_modalidad4.loc[i]=[len(juego.jugadores),juego.tiempo/60,juego.turno,juego.rondas,juego.turnos_perdidos]

In [ ]:
juegos_modalidad4.groupby('Cantidad de jugadores').agg({'Rondas':['mean','std'],'Tiempo':['mean','std'],'Turnos Totales':['mean','std'],'Turnos Perdidos Totales':['mean','std']}).reset_index()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, col in enumerate(['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=juegos_modalidad4, x=col, kde=True, multiple='stack', palette='muted', ax=ax)
    ax.set_title(f'Histograma de {col}')

plt.tight_layout()
plt.show()

Test de Pearson

In [ ]:


for col in ['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']:
    correlation, p_value = stats.pearsonr(juegos_modalida3[col], juegos_modalidad4[col])

    print(f"Coeficiente de Correlación: {correlation} {col}")
    print(f"P-valor: {p_value}")

    if p_value > 0.01:
        print("Las variables son independientes.")
    else:
        print("Las variables NO son independientes.")

Comparacion de medias

In [ ]:
for col in ['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']:
    t_stat, p_value = stats.ttest_ind(juegos_modalida3[col], juegos_modalidad4[col], equal_var=False)

    print("T-Statistic:", t_stat)
    print("P-Value:", p_value)

    alpha = 0.01
    
    if p_value < alpha:
        print("Rechazamos la hipótesis nula. Las medias son significativamente diferentes.")
    else:
        print("No se rechaza la hipótesis nula. Las medias no son significativamente diferentes.")

Kolmogorov Smirnov

In [ ]:
def ks_test(data, dist_name, dist_func):
    params = dist_func.fit(data)
    D, p_value = stats.kstest(data, dist_name, args=params)
    return D, p_value, params

distributions = {
    'NORM': stats.norm,
    'EXPON': stats.expon,
    'GAMMA': stats.gamma,
    'LOGNORM': stats.lognorm
}

for col in ['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']:
    print(f'{col.upper()}')
    for dist_name, dist_func in distributions.items():
        D, p_value, params = ks_test(juegos_modalida3[col], dist_name.lower(), dist_func)
        # print(f'\n{dist_name}')
        # print(f'Estadístico D: {D}')
        # print(f'P-valor: {p_value}')
        if p_value > 0.001:
            print(f"La variable {col} sigue una distribución {dist_name}.")
            print(p_value)
      
        

In [ ]:
for col in ['Rondas', 'Tiempo', 'Turnos Perdidos Totales', 'Turnos Totales']:
    print(f'{col.upper()}')
    for dist_name, dist_func in distributions.items():
        D, p_value, params = ks_test(juegos_modalidad4[col], dist_name.lower(), dist_func)
        # print(f'\n{dist_name}')
        # print(f'Estadístico D: {D}')
        # print(f'P-valor: {p_value}')
        if p_value > 0.001:
            print(f"La variable {col} sigue una distribución {dist_name}.")
            print(p_value)
      